In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from scipy.optimize import minimize
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
class HoltWinters:
    
    """
    Holt-Winters model with the anomalies detection using Brutlag method
    
    # series - initial time series
    # slen - length of a season
    # alpha, beta, gamma - Holt-Winters model coefficients
    # n_preds - predictions horizon
    # scaling_factor - sets the width of the confidence interval by Brutlag (usually takes values from 2 to 3)
    
    """
    
    
    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=2):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor
        
        
    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+1] - self.series[i]) / self.slen
        return sum / self.slen  
    
    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series)/self.slen)
        # calculate season averages
        print(n_seasons)
        for j in range(n_seasons):
            print(sum(self.series[self.slen*j : self.slen*j + self.slen]) / self.slen)
            print(",")
            season_averages.append(sum(self.series[self.slen*j : self.slen*j + self.slen]) / self.slen)
        # calculate initial values
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
            seasonals[i] = sum_of_vals_over_avg/n_seasons
        return seasonals   

          
    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBond = []
        self.LowerBond = []
        
        seasonals = self.initial_seasonal_components()
        
        for i in range(len(self.series)+self.n_preds):
            if i == 0: # components initialization
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i%self.slen])
                
                self.PredictedDeviation.append(0)
                
                self.UpperBond.append(self.result[0] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                
                self.LowerBond.append(self.result[0] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                continue
                
            if i >= len(self.series): # predicting
                m = i - len(self.series) + 1
                self.result.append((smooth + m*trend) + seasonals[i%self.slen])
                
                # when predicting increase uncertainty on each step
                self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 
                
            else:
                val = self.series[i]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append(smooth+trend+seasonals[i%self.slen])
                
                # Deviation using Brutlag algorithm.
                self.PredictedDeviation.append(self.gamma * np.abs(self.series[i] - self.result[i]) 
                                               + (1-self.gamma)*self.PredictedDeviation[-1])
                     
            self.UpperBond.append(self.result[-1] + 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.LowerBond.append(self.result[-1] - 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i%self.slen])


In [3]:
def timeseriesCVscore(params, series, loss_function=mean_squared_error, slen=24):
    """
        Returns error on CV  
        
        params - vector of parameters for optimization
        series - dataset with timeseries
        slen - season length for Holt-Winters model
    """
    # errors array
    errors = []
    
    values = series.values
    alpha, beta, gamma = params
    
    # set the number of folds for cross-validation
    tscv = TimeSeriesSplit(n_splits=3) 
    
    # iterating over folds, train model on each, forecast and calculate error
    for train, test in tscv.split(values):

        model = HoltWinters(series=values[train], slen=slen, 
                            alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
        model.triple_exponential_smoothing()
        
        predictions = model.result[-len(test):]
        actual = values[test]
        error = loss_function(predictions, actual)
        errors.append(error)
        
    return np.mean(np.array(errors))

In [4]:
df = pd.DataFrame({})
path = '/media/mayur/Softwares/BE Project/ProcessedDataset/November/'
for i in range(1,8):
    df_new = pd.read_csv(path+'sms-call-internet-mi-2013-11-0{}.csv'.format(i), parse_dates=['activity_date'])
    df = df.append(df_new)
    print("File " + str(i) + " added")
    
df['activity_hour'] += 24*(df.activity_date.dt.day-1)

series = df[df['square_id']==325]
series.set_index('activity_hour', inplace=True) 
series.drop(['square_id', 'activity_date'], axis=1, inplace=True)

File 1 added
File 2 added
File 3 added
File 4 added
File 5 added
File 6 added
File 7 added


/home/mayur/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [24]:
data = series[:125]
x = [0, 0, 0] 

# Minimizing the loss function (check doc - scipy.optimize)
opt = minimize(timeseriesCVscore, x0=x, 
               args=(data, mean_squared_error), 
               method="TNC", bounds = ((0, 1), (0, 1), (0, 1))
              )

# Take optimal values...
alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

# ...and train the model with them
model = HoltWinters(data, slen = 24, 
                    alpha = alpha_final, 
                    beta = beta_final, 
                    gamma = gamma_final, 
                    n_preds = 40, scaling_factor = 3)
model.triple_exponential_smoothing()

1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.701

3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712

,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.007

2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.523

1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.701

2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.52352922]
,
[4.70169345]
,
[4.00712329]
,
1
[4.52352922]
,
2
[4.52352922]
,
[4.70169345]
,
3
[4.523

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
def plotHoltWinters(series): #, plot_intervals=False, plot_anomalies=False):
    """
        series - dataset with timeseries
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 
    """
    
    plt.figure(figsize=(20, 10))
    plt.plot(model.result, label = "Model")
    plt.plot(series.values, label = "Actual")
    error = mean_absolute_percentage_error(series.values, model.result[:len(series)])
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))
    
    #if plot_anomalies:
    anomalies = np.array([np.NaN]*len(series))
    anomalies[series.values<model.LowerBond[:len(series)]] = \
        series.values[series.values<model.LowerBond[:len(series)]]
    anomalies[series.values>model.UpperBond[:len(series)]] = \
        series.values[series.values>model.UpperBond[:len(series)]]
    plt.plot(anomalies, "o", markersize=10, label = "Anomalies")
    
    #if plot_intervals:
    plt.plot(model.UpperBond, "r--", alpha=0.5, label = "Up/Low confidence")
    plt.plot(model.LowerBond, "r--", alpha=0.5)
    plt.fill_between(x=range(0,len(model.result)), y1=model.UpperBond, 
    y2=model.LowerBond, alpha=0.2, color = "grey")    
        
    plt.vlines(len(series), ymin=min(model.LowerBond), ymax=max(model.UpperBond), linestyles='dashed')
    plt.axvspan(len(series)-20, len(model.result), alpha=0.3, color='lightgrey')
    plt.grid(True)
    plt.axis('tight')
    plt.legend(loc="best", fontsize=13);
    
plotHoltWinters(data)